In [1]:
import sys
from flask import Flask, render_template, jsonify, redirect
# import pymongo
# import scrape_stock, scrape_covid
from yahoo_fin.stock_info import get_data
# from sqlalchemy import create_engine
import datetime
import pandas as pd
import requests

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
tickers = pd.DataFrame({
    'Sector' : ['Delivery', 'Delivery', 'Delivery', 'Medical Goods', 'Medical Goods', 'Medical Goods', 'Medical Services', 'Medical Services', 'Medical Services', 'Retail', 'Retail', 'Retail', 'Market', 'Market', 'Market'],
    'Name' : ['FedEx', 'UPS', 'DHL', 'Johnson & Johnson', 'Cardinal Health', '3M', 'Medical Imaging Corp', 'Community Health Systems', 'Teladoc Health', 'Walmart', 'Amazon', 'CVS', 'Dow Jones', 'S & P 500', 'Nasdaq'],
    'Ticker' : ['FDX', 'UPS', 'DPSGY', 'JNJ', 'CAH', 'MMM', 'MEDD', 'CYH', 'TDOC', 'WMT', 'AMZN', 'CVS', '^DJI', '^GSPC', '^IXIC'],
})


In [3]:
def scrape_stock_data(tick_name, start_date, end_date):
    df = get_data(ticker=tick_name, start_date=start_date, end_date=end_date)
    df.reset_index(inplace=True)
    df.rename(columns={'index' : 'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    #df['date'] = df['date'].dt.date
    return(df)

In [9]:
def build_stock_df(tickers, start_date, end_date):

    # get the ticker results for the time period
    for record in tickers.index:
        results = scrape_stock_data(tickers['Ticker'][record], start_date, end_date)
        results['company'] = tickers['Name'][record]
        results['sector'] = tickers['Sector'][record]
        
        #organize data for d3 use
        results['high'] = results.high.round(2)
        results['close'] = results.close.round(2)
        results['low'] = results.low.round(2)
        results['open'] = results.open.round(2)
        results['adjclose'] = results.adjclose.round(2)

        # if a combined dataset exists append these new results
        try:
            combined_results = combined_results.append(results, ignore_index=True)
            
        # if combined dataset doesn't exist make this dataframe  the combined results df
        except:
            combined_results = results

    return combined_results

In [5]:
def load_stock_data(tickers):
    """
    This functions chekcs the dates and gets new data if needed
    """
    
    # covid data starts on 1/22/20
    start_date = '01-22-2020'
    end_date = datetime.datetime.now().strftime('%m-%d-%Y')
    
    #end_date = '03-31-2020'
    
    # connect to the database
    connection_string = "postgres:postgres@localhost:5432/Project-2_db"
    engine = create_engine(f'postgresql://{connection_string}')
    
    # if the table exists set the start date to one day past the last date in the db
    if engine.has_table('stocks'):
        result = engine.execute('SELECT MAX (date) FROM stocks;')
        for row in result:
            start_date = (row[0] + datetime.timedelta(days = 1))
        
        # stock data is only available for weekdays
        while start_date.weekday() > 4:
            start_date = start_date + datetime.timedelta(days = 1)
        
        start_date = start_date.strftime('%m-%d-%Y')

    if start_date < end_date:
        stock_df = build_stock_df(tickers=tickers, 
                               start_date=start_date, 
                               end_date = end_date)
        
    
        stock_df.to_sql(name='stocks', con=engine, if_exists='append')

In [6]:
def get_covid_data(get_dates):
    
    stock_dates = pd.DataFrame(get_dates, columns=['date'])
#     stock_dates.rename(columns={'index' : 'date'}, inplace=True)
    stock_dates['date'] = pd.to_datetime(stock_dates['date'])
    url  = 'https://pomber.github.io/covid19/timeseries.json'
    response = requests.get(url).json()
    
    dateList = []
    confirmedList = []
    for i in response:

        #all records for a simgle country
        country_data = response[i]

        country_records = len(response[i])    
        for j in range(0, country_records):

            #pull and append each country's daily stats to lists
            date = country_data[j]['date']
            confirmed = country_data[j]['confirmed']

            dateList.append(date)
            confirmedList.append(confirmed)
    
    #consolidate list into a ditionary
    covid_dict = {'date': dateList,
                'confirmed': confirmedList}

    #convert dictionary into dataframe
    covid_df = pd.DataFrame.from_dict(covid_dict)

    #convert 'date' format
    covid_df['date'] = pd.to_datetime(covid_df['date'])
    #covid_df['date'] = covid_df['date'].dt.date

    #consolidate all records into a worlwide dataset
    covid_grouped = covid_df.groupby(['date'], as_index=False).sum()

    #sort date column
    covid_grouped = covid_grouped.sort_values(by=['date'])

    #calculate daily rate of change
    covid_grouped['delta'] = covid_grouped['confirmed'] - covid_grouped['confirmed'].shift(1)

    #convert NaN value to '0'
    covid_confirmed = covid_grouped.fillna(0)

    #remove columns that will not be used
    covid_data = covid_confirmed[['date','delta']]

    covid_data = pd.merge(stock_dates, covid_confirmed, on='date',how='inner')


    return covid_data
    

In [7]:
def load_covid_data():
    """
    This functions chekcs the dates and gets new covid data if needed
    """
    
    # connect to the database
    connection_string = "postgres:postgres@localhost:5432/Project-2_db"
    engine = create_engine(f'postgresql://{connection_string}')
    
    try:
        stock_dates = set([date[0] for date in engine.execute('SELECT date FROM stocks;')])
    except:
        print('no stock data')
        return
    
    # if the table exists get the dates we need
    if engine.has_table('covid'):
        covid_dates = set([date[0] for date in engine.execute('SELECT date FROM covid;')])
        need_dates = stock_dates.difference(covid_dates)
        if len(need_dates) == 0:
            return
    else:
        need_dates = stock_dates

    covid_df = get_covid_data(need_dates)
    covid_df.to_sql(name='covid', con=engine, if_exists='append')

In [10]:
load_stock_data(tickers)
load_covid_data()

In [59]:
def get_stock_from_db(tickers):
    connection_string = "postgres:postgres@localhost:5432/Project-2_db"
    engine = create_engine(f'postgresql://{connection_string}')
    
    # get the results from the datframe
    stock_df = pd.read_sql(sql='stocks', con=engine)
#     covid_df = pd.read_sql(sql='covid', con=engine)
    
    # build the stock data
    stock_dict = {}
    for ticker in tickers['Ticker'].iteritems():
        ticker_df = stock_df[stock_df['ticker'] == ticker[1]].sort_values('date')
   
        price_list = []
        for index, row in ticker_df.iterrows():
            prices = [row['high'],row['close'],row['low']]
            price_list.append(prices)
        price_list.insert(0,ticker[1])
        
        stock_dict.update({ticker[1] : price_list})
    
    date_list = ticker_df['date'].astype(str).tolist()
    date_list.insert(0,'date') 
    stock_dict.update({'date' : date_list})

    
    return(stock_dict)

stock_data = get_stock_from_db(tickers)

In [58]:
print(stock_data)

{'FDX': ['FDX', [158.85, 155.54, 155.47], [155.34, 154.88, 152.73], [155.64, 154.07, 152.31], [150.18, 148.49, 147.51], [149.95, 148.4, 147.11], [150.07, 148.06, 147.91], [148.4, 148.26, 143.36], [147.29, 144.64, 143.59], [147.4, 143.84, 143.6], [148.24, 147.51, 146.0], [150.85, 149.95, 148.82], [151.38, 148.63, 148.49], [157.44, 155.66, 146.42], [160.29, 156.78, 156.2], [159.56, 158.0, 156.87], [159.81, 158.51, 157.23], [161.83, 161.47, 156.48], [161.29, 158.62, 158.0], [160.91, 158.57, 156.39], [161.69, 161.14, 158.83], [165.25, 164.91, 159.5], [164.14, 163.25, 161.69], [157.72, 154.85, 152.62], [155.8, 145.6, 144.81], [149.39, 144.53, 143.56], [145.49, 138.76, 136.48], [141.2, 141.17, 134.06], [142.52, 139.75, 135.28], [142.17, 133.09, 132.56], [136.46, 136.41, 132.18], [133.14, 129.93, 128.75], [130.65, 127.68, 123.79], [121.58, 115.7, 113.85], [121.75, 120.67, 113.87], [118.3, 111.01, 108.53], [105.18, 97.0, 96.2], [106.68, 106.63, 96.0], [99.87, 90.49, 90.49], [100.15, 94.96, 88.

In [64]:
def get_covid_from_db():
    connection_string = "postgres:postgres@localhost:5432/Project-2_db"
    engine = create_engine(f'postgresql://{connection_string}')
    covid_df = pd.read_sql(sql='covid', con=engine).sort_values('date')
    
    covid_confirmed_list = covid_df['delta'].tolist()
    covid_confirmed_list.insert(0,"No. of New Cases")
    
    return([covid_confirmed_list])

In [65]:
covid_data = get_covid_from_db()

In [66]:
print(covid_data)

[['No. of New Cases', 0.0, 99.0, 287.0, 809.0, 2651.0, 588.0, 2068.0, 1693.0, 3094.0, 4011.0, 3743.0, 3159.0, 3597.0, 2612.0, 2040.0, 419.0, 15147.0, 6517.0, 1878.0, 503.0, 558.0, 622.0, 603.0, 845.0, 982.0, 1358.0, 1366.0, 1937.0, 2534.0, 2280.0, 2766.0, 3915.0, 3769.0, 5030.0, 7255.0, 2477.0, 16853.0, 14120.0, 15528.0, 17719.0, 27749.0, 29638.0, 41278.0, 39810.0, 49612.0, 61938.0, 63700.0, 62249.0, 75098.0, 74988.0, 80851.0, 82372.0, 71647.0, 74961.0, 83712.0, 85474.0, 70117.0, 70357.0, 79862.0, 96364.0, 87723.0, 71169.0, 76844.0, 74958.0, 84679.0, 102249.0]]


In [30]:
ticker_df = stock_df[[]]

index       date     open     high      low    close  adjclose  \
0        0 2020-01-22   157.49   158.85   155.47   155.54    154.76   
1        1 2020-01-23   155.00   155.34   152.73   154.88    154.11   
2        2 2020-01-24   155.64   155.64   152.31   154.07    153.30   
3        3 2020-01-27   150.11   150.18   147.51   148.49    147.75   
4        4 2020-01-28   149.75   149.95   147.11   148.40    147.66   
..     ...        ...      ...      ...      ...      ...       ...   
985    985 2020-04-20  8553.38  8684.91  8553.38  8560.73   8560.73   
986    986 2020-04-21  8460.69  8480.29  8215.69  8263.23   8263.23   
987    987 2020-04-22  8434.55  8537.31  8404.54  8495.38   8495.38   
988    988 2020-04-23  8528.84  8635.23  8475.20  8494.75   8494.75   
989    989 2020-04-24  8530.08  8642.93  8464.42  8634.52   8634.52   

         volume ticker company    sector  
0       1911100    FDX   FedEx  Delivery  
1       2059100    FDX   FedEx  Delivery  
2       1973600    FDX   FedEx  Delivery  
3       3368400    FDX   FedEx  Delivery  
4       2190000    FDX   FedEx  Delivery  
..          ...    ...     ...       ...  
985  3796950000  ^IXIC  Nasdaq    Market  
986  3756290000  ^IXIC  Nasdaq    Market  
987  3025060000  ^IXIC  Nasdaq    Market  
988  3734720000  ^IXIC  Nasdaq    Market  
989  3673170000  ^IXIC  Nasdaq    Market  

[990 rows x 11 columns]